<a href="https://colab.research.google.com/github/earthtennison/SuperAILevel2/blob/main/PredictiveMaintainance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Binary Classification</h1>
<h6>Predict if that certain cycle time, found Abnormally</h6>

In [ ]:
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import copy

# Setting seed for reproducibility
np.random.seed(1234)  
PYTHONHASHSEED = 0

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, LSTM

# define path to save model
model_path = 'binary_model.h5'

#Data preparation

In [ ]:
# !gdown --id xxxxxxxxxxxxxxxxxxxxxxxxxx

In [ ]:
if not os.path.isfile("/content/xxxxxxxxxxxxxx.xlsx"):
  !unzip /content/xxxxxxxxxxxxx.zip

In [ ]:
df = pd.read_excel('/content/xxxxxxxxxx.xlsx')
df

In [ ]:
# !gdown --id xxxxxxxxxxxxxxxxxxxx

In [ ]:
df_chin = pd.read_csv('/content/xxxxxxxxxxxxxxxxxxx.csv')
print(df_chin.groupby('Labels').count())

df_chin

#Data Preprocessing
Abnormally date:<br>
26/12/2019 15:13:00 PM<br>
18/02/2020 10:35:35 AM

Machine explode date (assume):<br>
12/01/2020 15:00:00 PM


###explore data

In [ ]:
df_temp = df[['date','VI3108101X.PV','VI3108101Y.PV','VI3108102X.PV','VI3108102Y.PV','VI3108131X.PV','VI3108131Y.PV','VI3108132X.PV','VI3108132Y.PV','VI3108151X.PV','VI3108151Y.PV','VI3108152X.PV','VI3108152Y.PV','VI3108161X.PV','VI3108161Y.PV','VI3108162X.PV','VI3108162Y.PV']]

df_temp1 = df_temp[
(df['VI3108101X.PV'] >= 6) &
(df['VI3108101Y.PV'] >= 6) &
(df['VI3108102X.PV'] >= 6) &
(df['VI3108102Y.PV'] >= 6) &
(df['VI3108131X.PV'] >= 6) &
(df['VI3108131Y.PV'] >= 6) &
(df['VI3108132X.PV'] >= 6) &
(df['VI3108132Y.PV'] >= 6) &
(df['VI3108151X.PV'] >= 6) &
(df['VI3108151Y.PV'] >= 6) &
(df['VI3108152X.PV'] >= 6) &
(df['VI3108152Y.PV'] >= 6) &
(df['VI3108161X.PV'] >= 6) &
(df['VI3108161Y.PV'] >= 6) &
(df['VI3108162X.PV'] >= 6) &
(df['VI3108162Y.PV'] >= 6)
]
df_temp.plot(x="date", kind="line", figsize=(20,8))
df_temp1.plot(x="date", kind="line", figsize=(20,8))

###chin's label

In [ ]:
df['label_chin'] = df_chin['Labels']
df

In [ ]:
df =df[
(df['VI3108101X.PV'] >= 6) &
(df['VI3108101Y.PV'] >= 6) &
(df['VI3108102X.PV'] >= 6) &
(df['VI3108102Y.PV'] >= 6) &
(df['VI3108131X.PV'] >= 6) &
(df['VI3108131Y.PV'] >= 6) &
(df['VI3108132X.PV'] >= 6) &
(df['VI3108132Y.PV'] >= 6) &
(df['VI3108151X.PV'] >= 6) &
(df['VI3108151Y.PV'] >= 6) &
(df['VI3108152X.PV'] >= 6) &
(df['VI3108152Y.PV'] >= 6) &
(df['VI3108161X.PV'] >= 6) &
(df['VI3108161Y.PV'] >= 6) &
(df['VI3108162X.PV'] >= 6) &
(df['VI3108162Y.PV'] >= 6)
]
df

In [ ]:
df['label_chin']

###put all 4 abnormal to df

In [ ]:
#manual 1th
start_man1 = '2018-01-01 12:00:00'
detect_man1 = '2018-08-06 09:00:00'
end_man1 = '2018-08-22 11:00:00'

#manual 2th
start_man2 = '2018-08-23 12:00:00'
detect_man2 = '2019-05-16 09:00:00'
end_man2 = '2019-07-19 07:00:00'

#abnormal 1th
start_ab1 = '2019-07-19 08:00:00'
detect_ab1 = '2019-12-26 15:00:00'
end_ab1 = '2020-01-12 15:00:00'

#abnormal 2th
start_ab2 = '2020-01-14 00:00:00'
detect_ab2 = '2020-02-18 10:00:00'
end_ab2 = '2020-03-04 00:00:00'


In [ ]:
### manual 1
mask = (df['date'] > start_man1) & (df['date'] <= end_man1)
manual_1 = df.loc[mask]

mask_label = (manual_1['date'] > detect_man1) & (manual_1['date'] <= end_man1)
label = mask_label.astype(int)

manual_1['label']=label
manual_1['id'] = 1
manual_1['cycle'] = [i for i in range(manual_1.shape[0])]

###manual 2
mask = (df['date'] > start_man2) & (df['date'] <= end_man2)
manual_2 = df.loc[mask]

mask_label = (manual_2['date'] > detect_man2) & (manual_2['date'] <= end_man2)
label_2 = mask_label.astype(int)

manual_2['label']=label_2
manual_2['id']=2
manual_2['cycle'] =[i for i in range(manual_2.shape[0])]

###abnorm 1
mask = (df['date'] > start_ab1) & (df['date'] <= end_ab1)
abnorm_1 = df.loc[mask]

mask_label = (abnorm_1['date'] > detect_ab1) & (abnorm_1['date'] <= end_ab1)
label_1 = mask_label.astype(int)

abnorm_1['label']=label_1
abnorm_1['id'] = 3
abnorm_1['cycle'] = [i for i in range(abnorm_1.shape[0])]

###abnorm 2
mask2 = (df['date'] > start_ab2) & (df['date'] <= end_ab2)
abnorm_2 = df.loc[mask2]

mask_label = (abnorm_2['date'] > detect_ab2) & (abnorm_2['date'] <= end_ab2)
label_2 = mask_label.astype(int)

abnorm_2['label']=label_2
abnorm_2['id']=4
abnorm_2['cycle'] =[i for i in range(abnorm_2.shape[0])]

print(manual_1.shape)
print(manual_2.shape)
print(abnorm_1.shape)
print(abnorm_2.shape)

In [ ]:
df_new = pd.concat([manual_1,manual_2,abnorm_1,abnorm_2],axis=0)
df_new.index = [i for i in range(df_new.shape[0])]
df_new

In [ ]:
df_new.columns

In [ ]:
df_new = df_new[[
       'date','id', 'cycle', 'FIC3104801.OP', 'FIC3104801.PV', 'FIC3104901.OP',
       'FIC3104901.PV', 'FIC3105201.OP', 'FIC3105201.PV', 'JI3104901.PV',
       'LI3104702.PV', 'LIC3105201.OP', 'LIC3105201.PV', 'LIC3105204.OP',
       'LIC3105204.PV', 'PI3100508.PV', 'PI3104801.PV', 'PI3104803.PV',
       'PI3104903.PV', 'PI3104905.PV', 'PIC3104701.OP', 'PIC3104701.PV',
       'PIC3104901.OP', 'SI3104901.PV', 'TI3104801.PV', 'TI3104802.PV',
       'TI3104901.PV', 'TI3104902.PV', 'TI3105203.PV', 'TI3105301.PV',
       'TI3108102.PV', 'TI3108104.PV', 'TI3108105.PV', 'TI3108106.PV',
       'TI3108107.PV', 'TI3108108.PV', 'TI3108111.PV', 'TI3108113.PV',
       'TI3108114.PV', 'TI3108115.PV', 'TI3108116.PV', 'TI3108119.PV',
       'TI3108131.PV', 'TI3108132.PV', 'TI3108134.PV', 'TI3108135.PV',
       'TI3108136.PV', 'VI3108101X.PV', 'VI3108101Y.PV', 'VI3108102X.PV',
       'VI3108102Y.PV', 'VI3108131X.PV', 'VI3108131Y.PV', 'VI3108132X.PV',
       'VI3108132Y.PV', 'VI3108151X.PV', 'VI3108151Y.PV', 'VI3108152X.PV',
       'VI3108152Y.PV', 'VI3108161X.PV', 'VI3108161Y.PV', 'VI3108162X.PV',
       'VI3108162Y.PV', 'ZI3108105A.PV', 'ZI3108105B.PV', 'ZI3108155A.PV',
       'ZI3108155B.PV', 'label','label_chin' 
]]
df_new

###RUL (remaining useful life)

In [ ]:
# Data Labeling - generate column RUL(Remaining Usefull Life or Time to Failure)
rul = pd.DataFrame(df_new.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
df_new = df_new.merge(rul, on=['id'], how='left')
df_new['RUL'] = df_new['max'] - df_new['cycle'] #####important!!!
df_new.drop('max', axis=1, inplace=True)
df_new

###Normalize

In [ ]:
# MinMax normalization (from 0 to 1)

df_new['cycle_norm'] = df_new['cycle']
cols_normalize = df_new.columns.difference(['date','id','cycle','RUL','label'])
print(cols_normalize)

min_max_scaler = preprocessing.MinMaxScaler()
norm_df = pd.DataFrame(min_max_scaler.fit_transform(df_new[cols_normalize]), 
                            columns=cols_normalize, 
                            index=df_new.index)
join_df = df_new[df_new.columns.difference(cols_normalize)].join(norm_df)
df_new = join_df.reindex(columns = df_new.columns)

df_new

###More modification
* predicted 7 days before (use label of 7 day forward)
* train separately each gang of sensors


In [ ]:
df_new['id']

In [ ]:
df_new_ab1 = df_new[(df_new['id']==1)|(df_new['id']==2)]
df_new_ab1['label_6'] = df_new_ab1['label'][6:]
df_new_ab1 = df_new_ab1.dropna().reset_index(drop=True)

df_new_ab1


In [ ]:

df_new_ab2 = df_new[(df_new['id']==3)|(df_new['id']==4)]
df_new_ab2['label_6'] = df_new_ab2['label'][6:]
df_new_ab2 = df_new_ab2.dropna().reset_index(drop=True)
df_new_ab2

In [ ]:
#compressor gang
compressor = [
'TI3108102.PV'
,'TI3108104.PV'
,'TI3108105.PV'
,'TI3108106.PV'
,'TI3108107.PV'
,'TI3108108.PV'
,'TI3108116.PV'
,'TI3108119.PV'
]

#motor left and right gang
motor_wing = [
'TI3108111.PV'
,'TI3108113.PV'
,'TI3108114.PV'
,'TI3108115.PV'
,'TI3108116.PV'
,'TI3108119.PV'
,'TI3108131.PV'
,'TI3108132.PV'
]

#inside motor gang
motor_in = [
'TI3108134.PV'
,'TI3108135.PV'
,'TI3108136.PV'

]

topTen = [
'LIC3105201.PV'
,'VI3108132Y.PV'
,'LIC3105204.PV'
,'FIC3105201.OP'
,'TI3108114.PV'
,'TI3105301.PV'
,'TI3108115.PV'
,'LIC3105204.OP'
,'VI3108161X.PV'
,'VI3108101Y.PV'
]

In [ ]:
df_com_1 = df_new_ab1[['date','id','cycle','RUL','cycle_norm']+compressor+['label','label_6']]
df_motor_wing_1 = df_new_ab1[['date','id','cycle','RUL','cycle_norm']+motor_wing+['label','label_6']]
df_motor_in_1 = df_new_ab1[['date','id','cycle','RUL','cycle_norm']+motor_in+['label','label_6']]
df_topTen_1 = df_new_ab1[['date','id','cycle','RUL','cycle_norm']+topTen+['label','label_6']]

df_com_2 = df_new_ab2[['date','id','cycle','RUL','cycle_norm']+compressor+['label','label_6']]
df_motor_wing_2 = df_new_ab2[['date','id','cycle','RUL','cycle_norm']+motor_wing+['label','label_6']]
df_motor_in_2 = df_new_ab2[['date','id','cycle','RUL','cycle_norm']+motor_in+['label','label_6']]
df_topTen_2 = df_new_ab2[['date','id','cycle','RUL','cycle_norm']+topTen+['label','label_6']]
df_topTen_2

#LSTM

###slicing window

In [ ]:
# pick a large window size of 50 cycles
sequence_length = 50

def input(df_temp,feature):
  
  # function to reshape features into (samples, time steps, features) 
  def gen_sequence(id_df, seq_length, seq_cols):
      """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
      we need to drop those which are below the window-length. An alternative would be to pad sequences so that
      we can use shorter ones """
      # for one id I put all the rows in a single matrix
      data_matrix = id_df[seq_cols].values
      num_elements = data_matrix.shape[0]
      # Iterate over two lists in parallel.
      # For example id1 have 192 rows and sequence_length is equal to 50
      # so zip iterate over two following list of numbers (0,142),(50,192)
      # 0 50 -> from row 0 to row 50
      # 1 51 -> from row 1 to row 51
      # 2 52 -> from row 2 to row 52
      # ...
      # 141 191 -> from row 141 to 191
      for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
          yield data_matrix[start:stop, :]
          
  # pick the feature columns 
  # sequence_cols = [
  #       'FIC3104801.OP', 'FIC3104801.PV', 'FIC3104901.OP',
  #       'FIC3104901.PV', 'FIC3105201.OP', 'FIC3105201.PV', 'JI3104901.PV',
  #       'LI3104702.PV', 'LIC3105201.OP', 'LIC3105201.PV', 'LIC3105204.OP',
  #       'LIC3105204.PV', 'PI3100508.PV', 'PI3104801.PV', 'PI3104803.PV',
  #       'PI3104903.PV', 'PI3104905.PV', 'PIC3104701.OP', 'PIC3104701.PV',
  #       'PIC3104901.OP', 'SI3104901.PV', 'TI3104801.PV', 'TI3104802.PV',
  #       'TI3104901.PV', 'TI3104902.PV', 'TI3105203.PV', 'TI3105301.PV',
  #       'TI3108102.PV', 'TI3108104.PV', 'TI3108105.PV', 'TI3108106.PV',
  #       'TI3108107.PV', 'TI3108108.PV', 'TI3108111.PV', 'TI3108113.PV',
  #       'TI3108114.PV', 'TI3108115.PV', 'TI3108116.PV', 'TI3108119.PV',
  #       'TI3108131.PV', 'TI3108132.PV', 'TI3108134.PV', 'TI3108135.PV',
  #       'TI3108136.PV', 'VI3108101X.PV', 'VI3108101Y.PV', 'VI3108102X.PV',
  #       'VI3108102Y.PV', 'VI3108131X.PV', 'VI3108131Y.PV', 'VI3108132X.PV',
  #       'VI3108132Y.PV', 'VI3108151X.PV', 'VI3108151Y.PV', 'VI3108152X.PV',
  #       'VI3108152Y.PV', 'VI3108161X.PV', 'VI3108161Y.PV', 'VI3108162X.PV',
  #       'VI3108162Y.PV', 'ZI3108105A.PV', 'ZI3108105B.PV', 'ZI3108155A.PV',
  #       'ZI3108155B.PV','cycle_norm'
                
  # ]


  sequence_cols = feature

  # generator for the sequences
  seq_gen = (list(gen_sequence(df_temp[df_temp['id']==id], sequence_length, sequence_cols)) 
            for id in df_temp['id'].unique())
  # generate sequences and convert to numpy array
  seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
  print("sequence")
  print(seq_array.shape)
  print(seq_array)


  # function to generate labels
  def gen_labels(id_df, seq_length, label):
      # For one id I put all the labels in a single matrix.
      # For example:
      # [[1]
      # [4]
      # [1]
      # [5]
      # [9]
      # ...
      # [200]] 
      data_matrix = id_df[label].values
      num_elements = data_matrix.shape[0]
      # I have to remove the first seq_length labels
      # because for one id the first sequence of seq_length size have as target
      # the last label (the previus ones are discarded).
      # All the next id's sequences will have associated step by step one label as target. 
      return data_matrix[seq_length:num_elements, :]

  # generate labels

  lab = ['label_6']
  label_gen = [gen_labels(df_temp[df_temp['id']== id], sequence_length, lab) for id in df_temp['id'].unique()]
  label_array = (np.concatenate(label_gen).astype(np.float32))
  # print("label")
  # print(label_array)
  # print(label_array.shape)

  return seq_array,label_array

###change input feature here!!!

In [ ]:
#change both df and fea
seq_array_ab1, label_array_ab1 = input(df_motor_wing_1, motor_wing)
seq_array_ab2, label_array_ab2 = input(df_motor_wing_2, motor_wing)

In [ ]:
label_array_ab1 = np.array(label_array_ab1)
print(label_array_ab1)
print(len(label_array_ab1))
print(label_array_ab1[0])
print(len(label_array_ab1[0]))

In [ ]:
label_array_ab1.shape

In [ ]:
# Next, we build a deep network. 
# The first layer is an LSTM layer with 100 units followed by another LSTM layer with 50 units. 
# Dropout is also applied after each LSTM layer to control overfitting. 
# Final layer is a Dense output layer with single unit and sigmoid activation since this is a binary classification problem.
# build the network
def train(seq_array,label_array):

  nb_features = seq_array.shape[2]
  nb_out = label_array.shape[1]
  sequence_length = 50

  model = Sequential()

  model.add(LSTM(
          input_shape=(sequence_length, nb_features),
          units=100,
          return_sequences=True))
  model.add(Dropout(0.2))

  model.add(LSTM(
            units=50,
            return_sequences=False))
  model.add(Dropout(0.2))

  model.add(Dense(units=nb_out, activation='sigmoid'))
#   model.add(Dense(units=2, activation='softmax'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  print(model.summary())

  # fit the network
  history = model.fit(seq_array, label_array, epochs=50, batch_size=200, validation_split=0.05, verbose=2,
            callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min'),
                        # keras.callbacks.ModelCheckpoint(filepath="weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5", monitor='val_loss', save_weights_only=True, save_best_only=True, mode='min', verbose=1)
                        keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='min', verbose=0)
                        ]
            )
  model.save('best_model.h5')

  # list all data in history
  print(history.history.keys())
  return model, history

###change train input here!!!

In [ ]:
model, history = train(seq_array_ab1,label_array_ab1)

#Evaluation val

In [ ]:
# summarize history for Accuracy
fig_acc = plt.figure(figsize=(10, 10))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
fig_acc.savefig("model_accuracy.png")

In [ ]:
# summarize history for Loss
fig_acc = plt.figure(figsize=(10, 10))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
fig_acc.savefig("model_loss.png")

###Predict

In [ ]:
def predict(model, seq_array,label_array):
  # training metrics

  scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)
  print('Accurracy: {}'.format(scores[1]))

  # make predictions and compute confusion matrix
  y_pred = model.predict_classes(seq_array,verbose=1, batch_size=200)
  y_true = label_array

  temp = pd.DataFrame(y_pred)
  temp.to_csv('binary_submit_train.csv', index = None)

  print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
  cm = confusion_matrix(y_true, y_pred)
  print(cm)

  # compute precision and recall
  precision = precision_score(y_true, y_pred)
  recall = recall_score(y_true, y_pred)
  print( 'precision = ', precision, '\n', 'recall = ', recall)
  print('\n')
  # Plot in red color the predicted data and in green color the
  # actual data to verify visually the accuracy of the model.
  fig_verify = plt.figure(figsize=(10, 5))
  plt.plot(y_pred, color="red")
  plt.plot(y_true, color="green")
  plt.title('prediction')
  plt.ylabel('value')
  plt.xlabel('row')
  plt.legend(['predicted', 'actual data'], loc='upper left')
  plt.show()
  fig_verify.savefig("model_verify.png")
  return y_pred

###change predict input here!!!

In [ ]:
prediction = predict(model, seq_array_ab2, label_array_ab2)

#Pipeline

In [ ]:
df_test = df #excel from BI
df_test['id']=1

df_test = df_test[
(df['VI3108101X.PV'] >= 6) &
(df['VI3108101Y.PV'] >= 6) &
(df['VI3108102X.PV'] >= 6) &
(df['VI3108102Y.PV'] >= 6) &
(df['VI3108131X.PV'] >= 6) &
(df['VI3108131Y.PV'] >= 6) &
(df['VI3108132X.PV'] >= 6) &
(df['VI3108132Y.PV'] >= 6) &
(df['VI3108151X.PV'] >= 6) &
(df['VI3108151Y.PV'] >= 6) &
(df['VI3108152X.PV'] >= 6) &
(df['VI3108152Y.PV'] >= 6) &
(df['VI3108161X.PV'] >= 6) &
(df['VI3108161Y.PV'] >= 6) &
(df['VI3108162X.PV'] >= 6) &
(df['VI3108162Y.PV'] >= 6)
]
df_test = df_test[['date','id']+motor_wing]

# mask = (df_test['date'] > '2019-07-19 04:00:00	') & (df_test['date'] <= '2020-03-04 23:00:00')
# df_test = df_test.loc[mask]

df_test[['date']+motor_wing].plot(x="date", kind="line", figsize=(20,8))



In [ ]:
df_test

In [ ]:
def pipeline(df_test):

  
  #preprocess

  # MinMax normalization (from 0 to 1)

  cols_normalize = df_test.columns.difference(['date','id'])
  print(cols_normalize)

  min_max_scaler = preprocessing.MinMaxScaler()
  norm_df = pd.DataFrame(min_max_scaler.fit_transform(df_test[cols_normalize]), 
                              columns=cols_normalize, 
                              index=df_test.index)
  join_df = df_test[df_test.columns.difference(cols_normalize)].join(norm_df)
  df_test = join_df.reindex(columns = df_test.columns)


  # function to reshape features into (samples, time steps, features) 
  def gen_sequence(id_df, seq_length, seq_cols):
      """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
      we need to drop those which are below the window-length. An alternative would be to pad sequences so that
      we can use shorter ones """
      # for one id I put all the rows in a single matrix
      data_matrix = id_df[seq_cols].values
      num_elements = data_matrix.shape[0]
      # Iterate over two lists in parallel.
      # For example id1 have 192 rows and sequence_length is equal to 50
      # so zip iterate over two following list of numbers (0,142),(50,192)
      # 0 50 -> from row 0 to row 50
      # 1 51 -> from row 1 to row 51
      # 2 52 -> from row 2 to row 52
      # ...
      # 141 191 -> from row 141 to 191
      for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
          yield data_matrix[start:stop, :]


  sequence_cols = motor_wing
  sequence_length = 50

  # generator for the sequences
  seq_gen = (list(gen_sequence(df_test[df_test['id']==id], sequence_length, sequence_cols)) 
            for id in df_test['id'].unique())
  # generate sequences and convert to numpy array
  seq_array_test = np.concatenate(list(seq_gen)).astype(np.float32)
  print("sequence")
  print(seq_array_test.shape)
  print(seq_array_test)

  # if best iteration's model was saved then load and use it
  model_path = '/content/best_model.h5'
  if os.path.isfile(model_path):
    model = load_model(model_path)

  y_pred = model.predict_classes(seq_array_test,verbose=1, batch_size=200)

  #save result
  temp = pd.DataFrame(y_pred)
  temp.to_csv('binary_submit_train.csv', index = None)

  # Plot in red color the predicted data
  # actual data to verify visually the accuracy of the model.
  fig_verify = plt.figure(figsize=(10, 5))
  plt.plot(y_pred, color="red")
  plt.title('prediction')
  plt.ylabel('value')
  plt.xlabel('row')
  plt.legend(['predicted'], loc='upper left')
  plt.show()
  fig_verify.savefig("model_verify.png")

  return y_pred

In [ ]:
prediction = pipeline(df_test)

In [ ]:
pred_list = []
for x in prediction:
  pred_list.append(x[0])
len(prediction)

In [ ]:
pred_list

In [ ]:
# def clean_pred(df_test):

# #find first date of each id and delete first 50 row
#   df_test = copy.deepcopy(df_test)

#   for i in df_test['id'].unique(): #3,4

#     temp = df_test[df_test['id']==i]
#     # print(temp.index[0])

#     index = [i for i in range(temp.index[0],temp.index[0]+50)]
#     # print(index)

#     df_test = df_test.drop(index)

#   return df_test

In [ ]:
# temp = df_new_ab2[df_new_ab2['id']==4]
# temp2 =temp[temp['label_6']==1]
# temp2['date'].iloc[0]

In [ ]:
# noti_date = []
# for i in df_new_ab2['id'].unique():
#   temp = df_new_ab2[df_new_ab2['id']==i]
#   noti_date.append(temp['date'].iloc[0])

In [ ]:
pred_date = df_test[50:]#clean slicing window
pred_date['predict'] = pred_list
# pred_date = pred_date[pred_date['predict']==1]

In [ ]:
pred_date = pred_date[['date','predict']]
pred_date["date"] = pd.to_datetime(pred_date ["date"])
pred_date

###check

In [ ]:
mask = (pred_date['date'] > '2017-05-01 04:00:00	') & (pred_date['date'] <= '2022-05-25 23:00:00')
pred_date_temp = pred_date.loc[mask]

fig_verify = plt.figure(figsize=(20, 5))
pred_date_temp = pred_date_temp.set_index('date')
lines = pred_date_temp['predict'].plot.line()


###Por's code

In [ ]:
import datetime 

notify = list()
for idx, row in pred_date.iterrows():
    if row["predict"] == 1:
        notify.append(row["date"])

notify_before_seven_days = [notify[0].to_pydatetime()]
out = list()
out.append(notify[0].to_pydatetime())

last_alert = notify[0].to_pydatetime()
cnt = 0
cnt_out = list()
for n in notify:
    if (n.to_pydatetime() - last_alert).total_seconds() > 86400:
        out.append(n.to_pydatetime())
        notify_before_seven_days.append(n.to_pydatetime() - datetime.timedelta(days=7))
        last_alert = n.to_pydatetime()
        cnt_out.append(cnt)
        cnt = 0
    else:
        cnt += 1
        last_alert = n.to_pydatetime()
cnt_out.append(cnt)

date_final = notify_before_seven_days[np.argmax(cnt_out)].strftime("%m/%d/%Y %H:%M:%S")
date_final2 = out[np.argmax(cnt_out)].strftime("%m/%d/%Y %H:%M:%S")

In [ ]:
date_final

In [ ]:
print('The predicted date for anomaly is')
print(date_final)
print('\n')

print('The notify date for anomaly is')
print(date_final2)

print('\n')
print('Sensors cause')
print(motor_wing)

In [ ]:
df_test

In [ ]:
df_new_ab2

In [ ]:


#preprocess
# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    # for one id I put all the rows in a single matrix
    data_matrix = id_df[seq_cols].values
    num_elements = data_matrix.shape[0]
    # Iterate over two lists in parallel.
    # For example id1 have 192 rows and sequence_length is equal to 50
    # so zip iterate over two following list of numbers (0,142),(50,192)
    # 0 50 -> from row 0 to row 50
    # 1 51 -> from row 1 to row 51
    # 2 52 -> from row 2 to row 52
    # ...
    # 141 191 -> from row 141 to 191
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_matrix[start:stop, :]


sequence_cols = motor_wing
sequence_length = 50

# generator for the sequences
seq_gen = (list(gen_sequence(df_test[df_test['id']==id], sequence_length, sequence_cols)) 
          for id in df_test['id'].unique())




# generate sequences and convert to numpy array
seq_array_test = np.concatenate(list(seq_gen)).astype(np.float32)
print("sequence")
print(seq_array_test.shape)
print(seq_array_test)





# if best iteration's model was saved then load and use it
model_path = '/content/best_model.h5'
if os.path.isfile(model_path):
  model = load_model(model_path)

y_pred = model.predict_classes(seq_array_test,verbose=1, batch_size=200)







#save result
temp = pd.DataFrame(y_pred)
temp.to_csv('binary_submit_train.csv', index = None)

# Plot in red color the predicted data
# actual data to verify visually the accuracy of the model.
fig_verify = plt.figure(figsize=(10, 5))
plt.plot(y_pred, color="red")
plt.title('prediction')
plt.ylabel('value')
plt.xlabel('row')
plt.legend(['predicted'], loc='upper left')
plt.show()
fig_verify.savefig("model_verify.png")

